Импортируем необходимые модули

In [ ]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
from plotly import tools
import plotly.graph_objs as go

Файл с данными: "Analytic_test". [Ссылка.](https://docs.google.com/spreadsheets/d/1YoYnKTWtTtOpAdUYDmaDvIJghn8CFBFI/edit?usp=share_link&ouid=115750805206343556774&rtpof=true&sd=true)

В нем в упорядоченном формате собраны данные о 4 компаниях: Red, Black, Green и Blue.

По компаниям предоставлены следующие данные:
*   *Hid* - human_id, зашифрованный персональный идентификатор пользователя компаний
*   *has_company_id* - наличие зарегистрированного аккаунта в компании
*   *company_gmv_total* - суммарный gmv пользователя в сервисе
*   *company_gvm_order_count* - количество успешных заказов пользователя в сервисе
*   *company_activity_count* - количество активных действий пользователя в сервисе
*   *company_android* - установленное у пользователя приложение компании на платформе android
*   *company_apple* - установленное у пользователя приложение компании на платформе Apple
*   *company_web* - установленное у пользователя приложение компании на ПК
*   *is_msk* - территориальное расположение в Москве




Загружаем датасет

In [ ]:
%%time
companies = pd.read_excel('Analytic_test.xlsx', header=1, index_col=0)
companies.head()

CPU times: user 26.3 s, sys: 272 ms, total: 26.5 s
Wall time: 29.7 s


,HID,active_Red,has_Green_id,has_Black_id,has_Blue_id,Blue_activity_count,Blue_gmv_total,Green_gvm_total,Green_gvm_order_count,Black_gvm_total,...,Blue_android,Red_apple,Green_apple,Black_apple,Blue_apple,is_msk,Red_web,Green_web,Black_web,Blue_web
0,00003c7fb5e57ce673f1801f3679f5c25,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00006b602533083e3bebfcbff08613931,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0000736c3f76339196cb1cc759b8363b,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00009bdcc5307b38efde8690b8bd658e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0000bc3186b7b31d9b35d3fbbb250b9f8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Preprocessing

In [ ]:
print('Shape of companies dataset:', companies.shape)

Shape of companies dataset: (100000, 24)


In [ ]:
companies['HID'].nunique()

100000

Все пользователи уникальны

In [ ]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   HID                    100000 non-null  object 
 1   active_Red             100000 non-null  float64
 2   has_Green_id           100000 non-null  float64
 3   has_Black_id           100000 non-null  float64
 4   has_Blue_id            100000 non-null  float64
 5   Blue_activity_count    100000 non-null  object 
 6   Blue_gmv_total         99999 non-null   float64
 7   Green_gvm_total        100000 non-null  float64
 8   Green_gvm_order_count  100000 non-null  object 
 9   Black_gvm_total        100000 non-null  float64
 10  Black_gvm_order_count  100000 non-null  object 
 11  Red_android            100000 non-null  float64
 12  Green_android          100000 non-null  float64
 13  Black_android          100000 non-null  float64
 14  Blue_android           100000 non-nul

Скорее всего фича active_Red должна называться has_Red_id.

Проверим это

In [ ]:
companies['active_Red'].unique()

array([0., 1.])

Так как в данной фиче содержатся 0 и 1, можно сделать вывод о том, что она отвечает за регистрацию аккаунта пользователя в компании Red

Переименуем даную фичу

In [ ]:
companies = companies.rename(columns={'active_Red': 'has_Red_id'})

Заменим в фичах company_gvm_total gvm на gmv

In [ ]:
companies = companies.rename(columns={'Green_gvm_total': 'Green_gmv_total',
                              'Black_gvm_total': 'Black_gmv_total'})

Есть одно значение NaN в Blue_gmv_total.

Посмотрим на него

In [ ]:
companies[companies['Blue_gmv_total'].isna()]

,HID,has_Red_id,has_Green_id,has_Black_id,has_Blue_id,Blue_activity_count,Blue_gmv_total,Green_gmv_total,Green_gvm_order_count,Black_gmv_total,...,Blue_android,Red_apple,Green_apple,Black_apple,Blue_apple,is_msk,Red_web,Green_web,Black_web,Blue_web
68383,18f2525eb8c1bbb35625163925171fb63cbb,0.0,1.0,0.0,3.0,4.0,NaN,820.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


Заменим пропущенное значение на 0, так как данная фича отвечает за gmv

In [ ]:
companies = companies.fillna(0)
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   HID                    100000 non-null  object 
 1   has_Red_id             100000 non-null  float64
 2   has_Green_id           100000 non-null  float64
 3   has_Black_id           100000 non-null  float64
 4   has_Blue_id            100000 non-null  float64
 5   Blue_activity_count    100000 non-null  object 
 6   Blue_gmv_total         100000 non-null  float64
 7   Green_gmv_total        100000 non-null  float64
 8   Green_gvm_order_count  100000 non-null  object 
 9   Black_gmv_total        100000 non-null  float64
 10  Black_gvm_order_count  100000 non-null  object 
 11  Red_android            100000 non-null  float64
 12  Green_android          100000 non-null  float64
 13  Black_android          100000 non-null  float64
 14  Blue_android           100000 non-nul

Приведем категориальные фичи к категориальному типу

In [ ]:
companies['has_Red_id'].unique()

array([0., 1.])

In [ ]:
companies['has_Green_id'].unique()

array([0., 1., 2.])

In [ ]:
companies['has_Green_id'].value_counts()

0.0    99114
1.0      882
2.0        4
Name: has_Green_id, dtype: int64

In [ ]:
companies['has_Black_id'].unique()

array([0., 1., 2.])

In [ ]:
companies['has_Black_id'].value_counts()

0.0    98921
1.0     1074
2.0        5
Name: has_Black_id, dtype: int64

In [ ]:
companies['has_Blue_id'].unique()

array([1., 0., 2., 3., 4.])

In [ ]:
companies['has_Blue_id'].value_counts()

0.0    86734
1.0    12749
2.0      501
3.0       14
4.0        2
Name: has_Blue_id, dtype: int64

In [ ]:
companies['Red_android'].unique()

array([0., 1.])

In [ ]:
companies['Green_android'].unique()

array([0., 1.])

In [ ]:
companies['Blue_android'].unique()

array([0., 1.])

In [ ]:
companies['Black_android'].unique()

array([0., 1.])

In [ ]:
companies['Red_apple'].unique()

array([0., 1.])

In [ ]:
companies['Green_apple'].unique()

array([0., 1.])

In [ ]:
companies['Blue_apple'].unique()

array([0., 1.])

In [ ]:
companies['Black_apple'].unique()

array([0., 1.])

In [ ]:
companies['is_msk'].unique()

array([0., 1.])

In [ ]:
companies['Red_web'].unique()

array([0., 1.])

In [ ]:
companies['Green_web'].unique()

array([0., 1.])

In [ ]:
companies['Blue_web'].unique()

array([0., 1.])

In [ ]:
companies['Black_web'].unique()

array([0., 1.])

Так как признаки has_company_id отвечают за наличие зарегистрированного аккаунта пользователя, то они должны быть представлены двумя значениями: 0 - нет аккаунта и 1 - есть аккаунт. Поэтому, если там находится значение больше 1 - то ставим 1.

In [ ]:
for col in companies.columns:
  if 'has' in col:
    companies.loc[companies[col] > 1.0, col] = 1.0

Создаем список категориальных фичей

In [ ]:
category_features_names = [col for col in companies.columns if ('has' in col) or
                           ('android' in col) or ('apple' in col) or ('msk' in col)
                           or ('web' in col)]

In [ ]:
companies[category_features_names] = companies[category_features_names].astype('category')

In [ ]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   HID                    100000 non-null  object  
 1   has_Red_id             100000 non-null  category
 2   has_Green_id           100000 non-null  category
 3   has_Black_id           100000 non-null  category
 4   has_Blue_id            100000 non-null  category
 5   Blue_activity_count    100000 non-null  object  
 6   Blue_gmv_total         100000 non-null  float64 
 7   Green_gmv_total        100000 non-null  float64 
 8   Green_gvm_order_count  100000 non-null  object  
 9   Black_gmv_total        100000 non-null  float64 
 10  Black_gvm_order_count  100000 non-null  object  
 11  Red_android            100000 non-null  category
 12  Green_android          100000 non-null  category
 13  Black_android          100000 non-null  category
 14  Blue_android         

Рассмотрим фичи с типом object

In [ ]:
companies['Blue_activity_count'].unique()

array(['2.0', '0.0', '1.0', '17.0', '13.0', '5.0', '6.0', '26.0', '8.0',
       '4.0', '27.0', '19.0', '12.0', '9.0', '3.0', '20.0', '10.0',
       '23.0', '7.0', '16.0', '14.0', '15.0', '53.0',
       datetime.datetime(2019, 5, 4, 0, 0), '32.0', '11.0', '18.0',
       '24.0', '119.0', '49.0', '39.0', '21.0',
       datetime.datetime(2019, 5, 7, 0, 0), '35.0', '72.0', '30.0',
       '71.0', '28.0', '42.0', '34.0', '41.0', '22.0', '29.0', '36.0',
       '90.0', '40.0', '43.0', datetime.datetime(2019, 5, 1, 0, 0),
       '51.0', '44.0', '25.0', '31.0', '45.0', '67.0', '33.0', '101.0',
       '74.0', '46.0', '111.0', '66.0', '38.0', '63.0', '100.0', '237.0',
       '59.0', datetime.datetime(2019, 5, 3, 0, 0),
       datetime.datetime(2019, 5, 6, 0, 0), '37.0', '47.0'], dtype=object)

In [ ]:
companies['Green_gvm_order_count'].unique()

array(['0.0', '2.0', '26.0', '5.0', '37.0', '3.0', '12.0', '7.0', '1.0',
       '6.0', '8.0', datetime.datetime(2019, 5, 4, 0, 0), '4.0', '40.0',
       '9.0', '22.0', '14.0', '15.0', '85.0', '25.0', '10.0', '20.0',
       '18.0', '11.0', '31.0', '23.0', '39.0', '13.0', '42.0', '127.0',
       '28.0', '101.0', '32.0', '30.0', '27.0', '51.0', '83.0', '36.0',
       '16.0', '64.0', '19.0', '29.0', '24.0', '52.0', '82.0', '45.0',
       '50.0', '62.0', '49.0', '21.0', '56.0', '65.0', '100.0', '76.0',
       '88.0', '17.0', '69.0', '114.0', '35.0', '125.0', '38.0', '108.0',
       '168.0', '84.0', '63.0', '75.0', '34.0', '78.0', '43.0', '60.0',
       datetime.datetime(2019, 5, 2, 0, 0)], dtype=object)

In [ ]:
companies['Black_gvm_order_count'].unique()

array(['0.0', '1.0', '4.0', '2.0', '3.0', '5.0', '8.0', '25.0', '7.0',
       '27.0', '6.0', '15.0', '9.0', '10.0', '50.0', '22.0', '13.0',
       '12.0', '14.0', '16.0', '20.0', '26.0', '11.0', '18.0', '21.0',
       datetime.datetime(2019, 5, 2, 0, 0), '19.0', '37.0', '33.0',
       '17.0', '35.0', '29.0', '51.0', '23.0', '42.0', '60.0', '34.0',
       '36.0', datetime.datetime(2019, 5, 6, 0, 0)], dtype=object)

Данные фичи содержат дату в качестве значения

Удалим строки, которые содержат дату и приведем столбцы с count к численному типу

In [ ]:
objs_features = ['Black_gvm_order_count', 'Green_gvm_order_count', 'Blue_activity_count']

In [ ]:
for feature in objs_features:
  companies[feature] = pd.to_numeric(companies[feature], errors='coerce')

In [ ]:
companies = companies.dropna()

In [ ]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99987 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   HID                    99987 non-null  object  
 1   has_Red_id             99987 non-null  category
 2   has_Green_id           99987 non-null  category
 3   has_Black_id           99987 non-null  category
 4   has_Blue_id            99987 non-null  category
 5   Blue_activity_count    99987 non-null  float64 
 6   Blue_gmv_total         99987 non-null  float64 
 7   Green_gmv_total        99987 non-null  float64 
 8   Green_gvm_order_count  99987 non-null  float64 
 9   Black_gmv_total        99987 non-null  float64 
 10  Black_gvm_order_count  99987 non-null  float64 
 11  Red_android            99987 non-null  category
 12  Green_android          99987 non-null  category
 13  Black_android          99987 non-null  category
 14  Blue_android           99987 non-null 

# Data Visualization

In [ ]:
companies.head()

,HID,has_Red_id,has_Green_id,has_Black_id,has_Blue_id,Blue_activity_count,Blue_gmv_total,Green_gmv_total,Green_gvm_order_count,Black_gmv_total,...,Blue_android,Red_apple,Green_apple,Black_apple,Blue_apple,is_msk,Red_web,Green_web,Black_web,Blue_web
0,00003c7fb5e57ce673f1801f3679f5c25,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00006b602533083e3bebfcbff08613931,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0000736c3f76339196cb1cc759b8363b,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00009bdcc5307b38efde8690b8bd658e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0000bc3186b7b31d9b35d3fbbb250b9f8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
companies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99987 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   HID                    99987 non-null  object  
 1   has_Red_id             99987 non-null  category
 2   has_Green_id           99987 non-null  category
 3   has_Black_id           99987 non-null  category
 4   has_Blue_id            99987 non-null  category
 5   Blue_activity_count    99987 non-null  float64 
 6   Blue_gmv_total         99987 non-null  float64 
 7   Green_gmv_total        99987 non-null  float64 
 8   Green_gvm_order_count  99987 non-null  float64 
 9   Black_gmv_total        99987 non-null  float64 
 10  Black_gvm_order_count  99987 non-null  float64 
 11  Red_android            99987 non-null  category
 12  Green_android          99987 non-null  category
 13  Black_android          99987 non-null  category
 14  Blue_android           99987 non-null 

Разделим данные о компаниях по разным таблицам

In [ ]:
red_company = pd.DataFrame(companies[['HID'] + [col for col in companies.columns
                                                if 'Red' in col] + ['is_msk']])
red_company.head()

,HID,has_Red_id,Red_android,Red_apple,Red_web,is_msk
0,00003c7fb5e57ce673f1801f3679f5c25,0.0,0.0,0.0,0.0,0.0
1,00006b602533083e3bebfcbff08613931,1.0,0.0,0.0,1.0,0.0
2,0000736c3f76339196cb1cc759b8363b,1.0,0.0,1.0,0.0,0.0
3,00009bdcc5307b38efde8690b8bd658e,0.0,0.0,0.0,1.0,0.0
4,0000bc3186b7b31d9b35d3fbbb250b9f8,1.0,0.0,1.0,0.0,0.0


In [ ]:
blue_company = pd.DataFrame(companies[['HID'] + [col for col in companies.columns
                                                if 'Blue' in col] + ['is_msk']])
blue_company.head()

,HID,has_Blue_id,Blue_activity_count,Blue_gmv_total,Blue_android,Blue_apple,Blue_web,is_msk
0,00003c7fb5e57ce673f1801f3679f5c25,1.0,2.0,0.0,0.0,0.0,0.0,0.0
1,00006b602533083e3bebfcbff08613931,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000736c3f76339196cb1cc759b8363b,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00009bdcc5307b38efde8690b8bd658e,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0000bc3186b7b31d9b35d3fbbb250b9f8,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
green_company = pd.DataFrame(companies[['HID'] + [col for col in companies.columns
                                                if 'Green' in col] + ['is_msk']])
green_company.head()

,HID,has_Green_id,Green_gmv_total,Green_gvm_order_count,Green_android,Green_apple,Green_web,is_msk
0,00003c7fb5e57ce673f1801f3679f5c25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00006b602533083e3bebfcbff08613931,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000736c3f76339196cb1cc759b8363b,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00009bdcc5307b38efde8690b8bd658e,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0000bc3186b7b31d9b35d3fbbb250b9f8,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
black_company = pd.DataFrame(companies[['HID'] + [col for col in companies.columns
                                                if 'Black' in col] + ['is_msk']])
black_company.head()

,HID,has_Black_id,Black_gmv_total,Black_gvm_order_count,Black_android,Black_apple,Black_web,is_msk
0,00003c7fb5e57ce673f1801f3679f5c25,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00006b602533083e3bebfcbff08613931,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0000736c3f76339196cb1cc759b8363b,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,00009bdcc5307b38efde8690b8bd658e,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0000bc3186b7b31d9b35d3fbbb250b9f8,0.0,0.0,0.0,0.0,0.0,0.0,0.0


График количества зарегистрированных пользователей для каждой компании

In [ ]:
name_companies = ['Red', 'Green', 'Blue', 'Black']
number_users = [companies['has_Red_id'].astype('int64').sum(),
                companies['has_Green_id'].astype('int64').sum(),
                companies['has_Blue_id'].astype('int64').sum(),
                companies['has_Black_id'].astype('int64').sum()]
fig = px.bar(x=name_companies, y=number_users, 
             title='Количество зарегистрированных пользователей в компаниях',
             labels={'y': 'Количество пользователей',
                     'x': 'Компания'})
fig.update_layout(title_x=0.5)
fig.show()

Больше всего пользователей зарегистрировано в компании Red (> 55000), далее идет компания Blue (~13000), в компаниях Green и Black зарегистрировано примерно по 1000 человек

По каждой компании посмотрим распределение по количеству установок в зависимости от платформы

In [ ]:
name_platform = ['Apple', 'Android', 'Web']

In [ ]:
fig = make_subplots(rows=2, cols=2,
                    subplot_titles=('Red Company', 'Green Company', 'Blue Company', 'Black Company'))

platform_users_red = [red_company['Red_apple'].astype('int64').sum(),
                      red_company['Red_android'].astype('int64').sum(),
                      red_company['Red_web'].astype('int64').sum()]

platform_users_green = [green_company['Green_apple'].astype('int64').sum(),
                      green_company['Green_android'].astype('int64').sum(),
                      green_company['Green_web'].astype('int64').sum()]

platform_users_blue = [blue_company['Blue_apple'].astype('int64').sum(),
                      blue_company['Blue_android'].astype('int64').sum(),
                      blue_company['Blue_web'].astype('int64').sum()]

platform_users_black = [black_company['Black_apple'].astype('int64').sum(),
                      black_company['Black_android'].astype('int64').sum(),
                      black_company['Black_web'].astype('int64').sum()]

fig.add_trace(
    go.Bar(x=name_platform, y=platform_users_red),
    row=1, col=1
)
fig.add_trace(
    go.Bar(x=name_platform, y=platform_users_green),
    row=1, col=2
)
fig.add_trace(
    go.Bar(x=name_platform, y=platform_users_blue),
    row=2, col=1
)
fig.add_trace(
    go.Bar(x=name_platform, y=platform_users_black),
    row=2, col=2
)
fig.update_layout(title_text='Количество установок в зависимости от платформы',
                  title_x=0.5, showlegend=False, yaxis_title='')
fig['layout']['yaxis']['title']='Количество установок'
fig['layout']['yaxis3']['title']='Количество установок'
fig.show()

У наиболее популярных компаний распределение скачиваний очень похожи, больше всего использование через ПК, далее идет Android и Apple.

Компания Green скорее всего сконцентрирована на использовании на мобильных устройствах (мобильное приложение). Так как количество установок на мобильных платформах сильно превышает ПК.

Компания Black в больше пользуется популярностью на Apple.

Проверим разницу в gvm между пользователями из Москвы и пользователями из других городов

In [ ]:
moscow_users = companies.groupby('is_msk').agg(mean_blue_gvm_total=('Blue_gmv_total', 'mean'),
                                mean_green_gvm_total=('Green_gmv_total', 'mean'),
                                mean_black_gvm_total=('Black_gmv_total', 'mean')).reset_index()
moscow_users

,is_msk,mean_blue_gvm_total,mean_green_gvm_total,mean_black_gvm_total
0,0.0,2.083343,13.125764,56.524174
1,1.0,5.254510,405.128304,376.784090


In [ ]:
feature_names=['Компания Blue', 'Компания Green', 'Компания Black']
fig = go.Figure(data=[
    go.Bar(name='Not Moscow', x=feature_names, y=moscow_users.iloc[0,][1:]),
    go.Bar(name='Moscow', x=feature_names, y=moscow_users.iloc[1,][1:])
])
fig.update_layout(title_text='Среднее значение GMV пользователей из Москвы и пользователей из других городов',
                  barmode='group', title_x=0.5, yaxis_title='Среднее значение GMV')
fig.show()

Из графика видно, что пользователи из Москвы совершают покупок на большие суммы (GMV) чем пользователи из других городов

Посмотрим на среднее значение gmv пользователей для каждой компании

In [ ]:
avg_blue_gvm = blue_company[blue_company['has_Blue_id'] == 1]['Blue_gmv_total'].mean()
avg_green_gvm = green_company[green_company['has_Green_id'] == 1]['Green_gmv_total'].mean()
avg_black_gvm = black_company[black_company['has_Black_id'] == 1]['Black_gmv_total'].mean()

In [ ]:
feature_names = ['Компания Blue', 'Компания Green', 'Компания Black']
fig = px.bar(x=feature_names, y=[avg_blue_gvm, avg_green_gvm, avg_black_gvm],
             title='Среднее значение gmv пользователей',
             labels={'y': 'Среднее значение gvm',
                    'x': 'Компания'})

fig.update_layout(title_x=0.5)
fig.show()

GMV Компании Blue гораздо ниже дргуих компаний, это может свидетельствовать о том, что в этой компании продаются в основном недорогие товары. Или компании Green и Black продают более нишевые и дорогие товары.

Исходя из среднего значения gvm и количества зарегистрированных пользователей, можно сделать вывод о том, что компании Green и Black продают более нишевые и дорогостоящие товары по сравнению с компанией Blue, поэтому у Green и Black гораздо меньше пользователей, но при этом гораздо больше среднее значение gvm.

Посмотрим на соотношение успешных заказов в компаниях Green и Black

In [ ]:
avg_green_order_count = green_company[green_company['has_Green_id'] == 1]['Green_gvm_order_count'].mean()
avg_black_order_count = black_company[black_company['has_Black_id'] == 1]['Black_gvm_order_count'].mean()

In [ ]:
feature_names = ['Компания Green', 'Компания Black']
fig = px.bar(x=feature_names, y=[avg_green_order_count, avg_black_order_count],
             title='Среднее значение успешных заказов пользователей',
             labels={'y': 'Среднее значение успешных заказов',
                    'x': 'Компания'})

fig.update_layout(title_x=0.5)
fig.show()

Исходя из графика сравнения среднего значения gvm пользователя и среденго значения успешных заказов пользователей, можно сделать вывод о том, что компания Black предоставляет более дорогие и нишевые продукты, поэтому при меньшем количестве успешных заказов, среднее значение gvm пользователя в компании Black больше по сравнению с Green.